In [ ]:
import os
import re
import math
import random
import zipfile
import collections
import datetime as dt
import urllib.request
import numpy as np
import tensorflow as tf


def download_textfile(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def collect_data():
    url = 'http://mattmahoney.net/dc/'
    filename = download_textfile('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    vocabulary_size = len(set(vocabulary))
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary, vocabulary_size

# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

# convert words to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

def Wiki_to_Onehot(WINDOW_SIZE):
    
    corpus_raw = "He is the king . The king is royal . She is the royal  queen "
    
    # convert to lower case
    corpus_raw = corpus_raw.lower()
    # use regular expression to extract words
    rgx_words = re.compile("([\w][\w']*\w)")
    words = rgx_words.findall(corpus_raw)
    # all duplicate words are removed
    words = set(words)
    
    # gives id to each unique word
    word2int = {}
    int2word = {}
    vocab_size = len(words) 
    for i,word in enumerate(words):
        word2int[word] = i
        int2word[i] = word

    # split raw sentences into a list of sentences.
    raw_sentences = corpus_raw.split('.')
    sentences = []
    for sentence in raw_sentences:
        if sentence != '':
            sentences.append(sentence.split())
            
    # contruct a list of word pairs as (input, label)
    data = []
    for sentence in sentences:
        for word_index, word in enumerate(sentence):
            for label in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
                if label != word:
                    data.append([word, label])
    
    # inputs and labels
    X_train = []
    y_train = []
    for data_word in data:
        X_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
        y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))

    # convert them to numpy arrays
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    
    return X_train, y_train

# adding layer function
def add_layer(name, inputs, in_size, out_size, Weights_initializer, activation, dropout_rate):
    Weights = tf.get_variable("Weights"+name, initializer=Weights_initializer, shape=[in_size, out_size], dtype=tf.float32)
    biases = tf.get_variable("biases"+name, initializer=tf.contrib.layers.xavier_initializer(), shape=[1, out_size], dtype=tf.float32)
    # input x weight + biases
    XW_plus_b = tf.matmul(inputs, Weights) + biases
    # dropout
    XW_plus_b = tf.nn.dropout(XW_plus_b, (1-dropout_rate))
    
    if activation is None:
        outputs = XW_plus_b
    else:
        outputs = activation(XW_plus_b)
    
    return outputs

def Word2Vec_model_fn(data):
    
    global vocabulary_size, embedding_size, batch_size, learning_rate, num_steps, num_skips, skip_window, valid_window, valid_size, reverse_dictionary
    
#     ########## Build Word2Vec network ##########
#     # define placeholder for inputs and labels
#     x = tf.placeholder(tf.float32, [None, X_train.shape[1]])
#     y = tf.placeholder(tf.float32, [None, y_train.shape[1]])
    
#     h1 = add_layer("h1", x, X_train.shape[1], EMBEDDING_DIM, tf.contrib.layers.variance_scaling_initializer(), None, dropout_rate)
#     outputs = add_layer("outputs", h1, EMBEDDING_DIM, X_train.shape[1], tf.contrib.layers.variance_scaling_initializer(), tf.nn.softmax, 0.0)
    
#     # define cross entropy loss function:
#     cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=y))
    
#     # training iteration and define Adam optimizer with learning rate
#     optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
#     ########## Build Word2Vec network ##########
    
#     ########## Training ##########
#     # start TF session
#     sess = tf.Session()
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for _ in range(training_iters):
#         sess.run(optimizer, feed_dict={x: X_train, y: y_train})
#         print('loss is : ', sess.run(cross_entropy, feed_dict={x: X_train, y: y_train}))
    
    
#     sess.close()
#     ########## Training ##########

    ########## Build Word2Vec network ##########
    
    # define placeholder for inputs and labels
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Look up embeddings for inputs.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the softmax
    weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases
    
    # convert train_context to a one-hot format
    train_one_hot = tf.one_hot(train_context, vocabulary_size)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm

    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)

    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    ########## Build Word2Vec network ##########
    
    ########## Training ##########
    # Add variable initializer.
    init = tf.global_variables_initializer()
    
    with tf.Session() as session:
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                    # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
        final_embeddings = normalized_embeddings.eval()
    
    ########## Training ##########
    

########## Define hyper-parameters ##########

WINDOW_SIZE = 2
EMBEDDING_DIM = 5
learning_rate = 0.1
training_iters = 10000

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a context
num_steps = 50000

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 20     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.

########## Define hyper-parameters ##########    

#X_train, y_train = Wiki_to_Onehot(WINDOW_SIZE)

data_index = 0
data, count, dictionary, reverse_dictionary, vocabulary_size = collect_data()
Word2Vec_model_fn(data)


Found and verified text8.zip
Initialized
Average loss at step  0 :  12.5501880646
Nearest to new: underhook, fatay, nomenclator, viviane, tzarina, leckebusch, demartinville, kamagraphy,
Nearest to american: andreesen, voidmage, slaveholders, pinakes, peter, saxicola, tabulator, hydration,
Nearest to years: lobbies, prosecutor, linus, tsukuba, worked, phixers, owi, polychora,
Nearest to by: carqueja, curseperl, normalize, spurts, jabao, francony, briggate, cubas,
Nearest to war: gelonins, sushi, appeased, ryukyuans, pyy, madesani, flaviceps, medley,
Nearest to three: vmware, ndendeule, regardles, upgradeability, canfield, cancri, guterson, hanjourin,
Nearest to UNK: snooping, kundur, jandia, poppped, footpegs, centralizes, xtree, ent,
Nearest to five: ashrams, longden, firchie, kamchatkan, salvus, steles, kaluga, forgiving,
Nearest to may: teiman, coiban, cyclooctene, naiskos, biliono, vunjo, halima, wordings,
Nearest to about: fanclubs, blockages, calydon, squirted, amahoney, ibadat, s